In [ ]:
# Importação de bibliotecas
from collections import defaultdict
import csv

import igraph as ig

In [ ]:
# Algoritmo de Johnson (identificação de ciclos)
MIN_SCC_SIZE = 2

def simple_cycles_ig(G: ig.Graph,
                     limit_len: int = -1,
                     limit_node_type: str = None,
                     log_file: str = None):
    """
        Rewrite of simple_cycles function from the networkx library to igraph library.

        This is a nonrecursive, iterator/generator version of Johnson's
        algorithm [1]_.  There may be better algorithms for some cases [2]_ [3]_.

        Parameters
        ----------
        G : igraph.Graph
           A directed graph
        limit_len : int
           Limit count of the nodes in cycle. If -1  (default), cycle length is unlimited.
        limit_node_type : string
           Node type to consider in cycle limit length. If None (default), consider all node types.
        log_file : string
           Log file path.

        Returns
        -------
        cycle_generator: generator
           A generator that produces elementary cycles of the graph.
           Each cycle is represented by a list of nodes along the cycle.

        References
        ----------
        .. [1] Finding all the elementary circuits of a directed graph.
           D. B. Johnson, SIAM Journal on Computing 4, no. 1, 77-84, 1975.
           http://dx.doi.org/10.1137/0204007
        .. [2] Enumerating the cycles of a digraph: a new preprocessing strategy.
           G. Loizou and P. Thanish, Information Sciences, v. 27, 163-182, 1982.
        .. [3] A search strategy for the elementary cycles of a directed graph.
           J.L. Szwarcfiter and P.E. Lauer, BIT NUMERICAL MATHEMATICS,
           v. 16, no. 2, 192-204, 1976.
    """

    def _unblock(thisnode, blocked, B):
        stack = {thisnode}
        while stack:
            node = stack.pop()
            if node in blocked:
                blocked.remove(node)
                stack.update(B[node])
                B[node].clear()

    def _get_neighbors(subG, node_id):
        return [subG.vs[i]['id']
                for i in subG.neighbors(subG.vs.find(id=node_id), mode='out')]

    def _strongly_connected_components(subG: ig.Graph, min_scc_size: int = 1):
        scc_list = list(subG.components(mode='STRONG'))  # list of list
        scc_list_obj_id = [subG.vs[scc]['id'] for scc in scc_list if
                           len(scc) >= min_scc_size]  # list of list
        return [set(frozenset(scc)) for scc in scc_list_obj_id]  # list of set

    def is_path_in_limit(subG, path, limit_len, limit_node_type):
        if limit_len < 0:
            return True
        elif limit_node_type is None:
            return len(path) <= limit_len
        else:
            node_type_count = 0
            for node_id in path:
                if subG.vs.find(id=node_id)['tipo'] == limit_node_type:
                    node_type_count += 1
                    if node_type_count > limit_len:
                        return False
            return True

    if not (isinstance(G, ig.Graph) and G.is_directed()):
        raise Exception('[simple_cycles_ig] G parameter '
                        'is not a instance of igraph.Graph class.')

    try:
        limit_len = int(limit_len)
    except ValueError:
        raise Exception(f"[simple_cycles_ig] limit '{limit_len}' "
                        f"is not a int type parameter.")

    # Johnson's algorithm requires some ordering of the nodes.
    # We assign the arbitrary ordering given by the strongly connected comps
    # There is no need to track the ordering as each node removed as processed.
    # Also we save the actual graph so we can mutate it. We only take the
    # edges because we do not want to copy edge and node attributes here.
    subG = G.copy()
    subG.vs['id'] = [v.index for v in subG.vs]
    total_vs = len(subG.vs)
    total_es = len(subG.es)
    # sccs = _strongly_connected_components(subG)
    sccs = _strongly_connected_components(subG=subG, min_scc_size=MIN_SCC_SIZE)
    i = 0
    while sccs:
        i += 1
        scc = sccs.pop()

        # order of scc determines ordering of nodes
        startnode = scc.pop()
        # Processing node runs "circuit" routine from recursive version
        path = [startnode]
        blocked = set()  # vertex: blocked from search?
        closed = set()  # nodes involved in a cycle
        blocked.add(startnode)
        B = defaultdict(set)  # graph portions that yield no elementary circuit
        stack = [(startnode, _get_neighbors(subG, startnode))]  # subG gives comp nbrs
        while stack:
            thisnode, nbrs = stack[-1]
            if nbrs and is_path_in_limit(subG, path, limit_len, limit_node_type):
                nextnode = nbrs.pop()
                if nextnode == startnode:
                    yield path[:]
                    closed.update(path)
                elif nextnode not in blocked:
                    path.append(nextnode)
                    stack.append((nextnode, _get_neighbors(subG, nextnode)))
                    closed.discard(nextnode)
                    blocked.add(nextnode)
                    continue
            # done with nextnode... look for more neighbors
            if (not nbrs) or (not is_path_in_limit(subG, path, limit_len, limit_node_type)):
                if thisnode in closed:
                    _unblock(thisnode, blocked, B)
                else:
                    for nbr in _get_neighbors(subG, thisnode):
                        if thisnode not in B[nbr]:
                            B[nbr].add(thisnode)
                stack.pop()
                path.pop()
        # done processing this node
        subG.delete_vertices(subG.vs.find(id=startnode).index)

        scc_subG_id = [subG.vs.find(id=node_id).index for node_id in scc]
        H = subG.subgraph(scc_subG_id)
        scc_extend_list = _strongly_connected_components(H)

        for scc_ext in scc_extend_list:
            if len(scc_ext) >= MIN_SCC_SIZE:
                sccs.extend([scc_ext])
            else:
                subG.delete_vertices([subG.vs.find(id=node_id).index for node_id in scc_ext])

In [ ]:
# Cria o Grafo
with open('sample.csv', mode = 'r', encoding = 'utf-8-sig') as csv_edges:
    dict_edges = csv.DictReader(csv_edges, delimiter = ';')
    graph = ig.Graph.DictList(vertices = None, edges = dict_edges, directed = True)
    
# Define novas propriedades tipo para os vértices
v_coresdict = {'C':'#007FFF', 'F':'#A66EA6', 'E':'#00FF00', 'S':'#F69E00', 'T':'#FFCE75'}
graph.vs['tipo'] = [name.split('-')[0] for name in graph.vs['name']]
graph.vs['label'] = graph.vs['name']

# Define novas propriedades tipo para as arestas
e_labeldict = {'C-F': '', 'E-C-FISC': 'Fiscal', 'E-C-GC': 'GC', 'E-C-SUP': 'Sup', 'F-S': '', 'F-T': '', 'P-E-CONJUGE': 'Cônjuge', 'P-E-FILHO_A': 'Filho(a)', 'P-E-IRMAO_A': 'Irmâ(o)', 'P-E-MAE': 'Mãe', 'P-E-PAI': 'Pai'}
graph.es['label'] = [e_labeldict[name] for name in graph.es['tipo']]
    
print(f"Grafo criado ({len(graph.vs)} vértices, {len(graph.es)} arestas)")

In [ ]:
# Visualiza o grafo
v_cores = [v_coresdict.get(vs['tipo'], '#FFFFFF') for vs in graph.vs]

ig.plot(graph, 
        bbox = (0, 0, 700, 700), 
        margin = 60,
        vertex_size = 30,
        vertex_color = v_cores,
        edge_color = 'black',
        layout = graph.layout_lgl())
#         layout = graph.layout_kamada_kawai())

In [ ]:
# Busca os ciclos
ciclos = []
limite_len = 8
for ciclo in simple_cycles_ig(graph, limite_len):
     ciclos.append(ciclo)
        
print(f"{len(ciclos)} ciclos (com até {limite_len} vértices) identificados")

In [ ]:
# Exemplo 1 (Parentesco Direto): Visualização
i = 2
ciclo = ciclos[i]

subG = graph.subgraph(ciclo)

v_coresdict = {'C':'#007FFF', 'F':'#A66EA6', 'E':'#00FF00', 'S':'#F69E00', 'T':'#FFCE75'}
v_cores = [v_coresdict.get(vs['tipo'], '#FFFFFF') for vs in subG.vs]

print('Dados Simulados')

ig.plot(subG, 
        bbox = (0, 0, 500, 300), 
        margin = (100, 50, 100, 50),
        vertex_size = 30,
        vertex_color = v_cores,
        edge_color = 'black',
        layout = subG.layout_lgl())

In [ ]:
# Exemplo 2 (Parentesco Direto): Visualização
i = 13
ciclo = ciclos[i]

subG = graph.subgraph(ciclo)

v_coresdict = {'C':'#007FFF', 'F':'#A66EA6', 'E':'#00FF00', 'S':'#F69E00', 'T':'#FFCE75'}
v_cores = [v_coresdict.get(vs['tipo'], '#FFFFFF') for vs in subG.vs]

print('Dados Simulados')

ig.plot(subG, 
        bbox = (0, 0, 500, 300), 
        margin = (100, 50, 100, 50),
        vertex_size = 30,
        vertex_color = v_cores,
        edge_color = 'black',
        layout = subG.layout_lgl())

In [ ]:
# Exemplo 3 (Parentesco Cruzado): Visualização
i = 14
ciclo = ciclos[i]

subG = graph.subgraph(ciclo)

v_coresdict = {'C':'#007FFF', 'F':'#A66EA6', 'E':'#00FF00', 'S':'#F69E00', 'T':'#FFCE75'}
v_cores = [v_coresdict.get(vs['tipo'], '#FFFFFF') for vs in subG.vs]

print('Dados Simulados')

ig.plot(subG, 
        bbox = (0, 0, 500, 300),         
        margin = 50,
        vertex_size = 30,
        vertex_color = v_cores,
        edge_color = 'black',
        layout = subG.layout_lgl())